In [66]:
import pandas as pd
import numpy as np

#需要优化的量

#关系矩阵
#dts
#M1=np.zeros((19,106),dtype=int) #进口：对应1
#M3=np.zeros((19,106),dtype=int) #国产：对应2
#ctd
#M2=np.zeros((1,18),dtype=int) #进口
#M4=np.zeros((1,18),dtype=int) #国产

#仓库状态矩阵
#O=np.zeros((20,1),dtype=int)  #是否开仓
#Z=np.zeros((20,1),dtype=int)  #是否智能


In [67]:
#参数导入
#国产，对应2
Q_dm=pd.read_csv('data_p/Q_dm.csv')
Q_dm=Q_dm.to_numpy()
Q_dm=Q_dm[:,1]
Q_dm=Q_dm.reshape((1,106))
Q_dm.shape

(1, 106)

In [68]:
#港口，对应1
Q_im=pd.read_csv('data_p/Q_im.csv')
Q_im=Q_im.to_numpy()
Q_im=Q_im[:,1]
Q_im=Q_im.reshape((1,106))
Q_dm.shape

(1, 106)

In [69]:
L_dts=pd.read_csv('data_p/d_matrix_dts.csv')
L_dts=L_dts.to_numpy()
L_dts=L_dts[1:]
L_dts=L_dts[:,2:]
L_dts_1=np.delete(L_dts,11,axis=0)  #im(delete neilu)
#k = 10 
#row_to_move = L_dts_1[k]
#L_dts_1 = np.delete(L_dts_1, k, axis=0)
#L_dts_1 = np.vstack([row_to_move, L_dts_1])

L_dts_2=np.delete(L_dts,6,axis=0)  #dm(delete gangkou)
#k = 6 
#row_to_move = L_dts_2[k]
#L_dts_2 = np.delete(L_dts_2, k, axis=0)
#L_dts_2 = np.vstack([row_to_move, L_dts_2])
L_dts_2.shape

(19, 106)

In [70]:
T_dts=pd.read_csv('data_p/t_matrix_dts.csv')
T_dts=T_dts.to_numpy()
T_dts=T_dts[1:]
T_dts=T_dts[:,2:]
T_dts_1=np.delete(T_dts,11,axis=0)#im(delete neilu)
T_dts_2=np.delete(T_dts,6,axis=0)#dm(delete gangkou)
T_dts_1

array([[532.0, 167.0, 491.0, ..., 578.0, 1478.0, 364.0],
       [1376.0, 959.0, 1295.0, ..., 556.0, 644.0, 935.0],
       [1639.0, 1209.0, 1163.0, ..., 962.0, 1130.0, 1253.0],
       ...,
       [1169.0, 758.0, 1140.0, ..., 310.0, 877.0, 709.0],
       [1189.0, 753.0, 651.0, ..., 501.0, 1249.0, 816.0],
       [948.0, 406.0, 859.0, ..., 175.0, 1137.0, 440.0]], dtype=object)

In [71]:
L_ctd=pd.read_csv('data_p/d_matrix_ctd.csv')
L_ctd=L_ctd.to_numpy()
L_ctd=L_ctd[:,2:]

In [72]:
T_ctd=pd.read_csv('data_p/t_matrix_ctd.csv')
T_ctd=T_ctd.to_numpy()
T_ctd=T_ctd[:,2:]

In [73]:
Pf=pd.read_csv('data_p/Pf.csv')
Pf=Pf.to_numpy()
Pf=Pf[:,1]
Pf.shape

(20,)

In [74]:
Of=pd.read_csv('data_p/Of.csv')
Of=Of.to_numpy()
Of=Of[:,1]
#Of.reshape((1,20))
Of.shape

(20,)

In [75]:
Cap=pd.read_csv('data_p/Cap.csv')
Cap=Cap.to_numpy()
Cap=Cap[:,1]
#Cap.reshape((1,20))
Cap.shape

(20,)

In [76]:
#约束条件
##时间
Q_all=Q_im.sum()
#改为返回耗时矩阵
def time_point(M_im,M_dm,T_dts_1,T_dts_2):
    M_im=M_im.tolist()
    M_im=np.array(M_im)
    M_dm=M_dm.tolist()
    M_dm=np.array(M_dm)
    #Q_out=0
    tmp1=M_im*T_dts_1
    tmp2=M_dm*T_dts_2  #这里改到对应的时间矩阵
    #print(tmp1,"\n",tmp2)
    tmp1=tmp1.sum(axis=0)
    tmp2=tmp2.sum(axis=0)
    #print(tmp1.shape)
    for i in range(0,106):
        if tmp1[i]>=10:
            Q_out+=Q_im[0,i]
        if tmp2[i]>=10:
            Q_out+=Q_dm[0,i]
    out_point=Q_out/Q_all
    return out_point
    #return tmp1,tmp2

def time_lim(M_im,M_dm):
    if time_point(M_im,M_dm)<=0.05:
        return True
    return False

#最大订单量不超过最大承载量（改为返回订单量矩阵）
def cap_lim(M_im,M_dm,Q_im,Q_dm):
    M_im=M_im.tolist()
    M_im=np.array(M_im)
    M_dm=M_dm.tolist()
    M_dm=np.array(M_dm)
    tmp=M_im*Q_im
    tmp1=tmp.sum(axis=1)
    tmp1=np.insert(tmp1,10,values=0)
    tmp2=M_dm*Q_dm
    tmp3=tmp2.sum(axis=1)
    tmp3=np.insert(tmp3,6,values=0)
    sum=tmp1+tmp3
    """
    s1=0
    s2=0
    for j in range(0,20):
        if j==6:
            sum=tmp1[s1]
            s2-=1
        elif j==11:
            sum=tmp3[s2]
            s1-=1
        else:
            sum=tmp1[s1]+tmp3[s2]
        if sum>Cap[j]:
            return False
        s1+=1
        s2+=1
    """
    return sum

#cdc必须开(不需要)
def cdc_op(O):
    return O[6]==1 and O[11]==1

#每个门店对应一个商店(不需要)
def sd_lim(M_1,M_3):
    M1=M_1.tolist()
    M1=np.array(M1)
    M3=M_3.tolist()
    M3=np.array(M3)
    M1_s=M1.sum(axis=0)
    #print(M1_s.shape)
    M2_s=M3.sum(axis=0)
    #print(M2_s)
    flag=True
    for i in M1_s:
        if i==1:
            flag=True
        else:
            flag=False

    for i in M2_s:
        if i==1:
            flag=True
        else:
            flag=False
    return flag



In [77]:
#目标优化式
##处置成本
def ccl_cp(Z,Q,M_s,M_d,k,Of,Pf):
    Q_dc=M_s*Q  
    Q_dc=Q_dc.sum(axis=1)
    #print(tmp1) #19,一维数组
    #tmp1=tmp1.reshape((1,19))
    Q_rdc=np.delete(Q_dc,k)
    Pf=np.delete(Pf,k)
    Of=np.delete(Of,k)
    Z=np.delete(Z,k)
    #print(tmp11) 
    Q_cdc=M_d*Q_rdc
    Sum_cdc=Q_cdc.sum()
    m=-0.5*Z+1
    Sum_rdc=Of+Q_dc*Pf*m+Z
    Sum_rdc=Sum_rdc.sum()
    return Sum_rdc+Sum_cdc

#运输成本
def ccl_ct(k,Q,M_s,M_d,L_s,L_d):
    #ct1=np.zeros((1,20))
    sum=0
    tmp1 = M_s * Q * L_s * 1.25
    Q_c = Q @ M_s.T
    Q_c = np.delete(Q_c,k,axis=1) 
    tmp2= M_d * Q_c * L_d.T * 0.6
    sum+=tmp1.sum()+tmp2.sum()
    return sum

#总成本
def cal_all(M1,M2,M3,M4,O,Z,Of,Pf,L_ctd,L_dts_1,L_dts_2):
    M1=M1.tolist()
    M1=np.array(M1)
    M2=M2.tolist()
    M2=np.array(M2)
    M3=M3.tolist()
    M3=np.array(M3)
    M4=M4.tolist()
    M4=np.array(M4)
    O=O.tolist()
    O=np.array(O)
    Z=Z.tolist()
    #print(Z[0])
    Z=np.array(Z)
    c=0
    #进口
    c+=ccl_cp(Z,Q_im,M1,M2,10,Of,Pf)
    #print(c)
    c+=ccl_ct(10,Q_im,M1,M2,L_dts_1,L_ctd[0])
    #print(c)
    #国产
    c+=ccl_cp(Z,Q_dm,M3,M4,6,Of,Pf)
    #print(c)
    c+=ccl_ct(6,Q_dm,M3,M4,L_dts_2,L_ctd[1])
    #print(c)
    return c

In [78]:
import gurobipy as gp
from gurobipy import GRB

#2 个cdc
cdc_cnt=2
# 106 个store
s_cnt=106
#18个rdc
rdc_cnt=18
cdc_cnt_1=19
dc_cnt=20

# 创建模型
m = gp.Model("ass_mov")
# 创建变量
# 定义 BigM
BigM = 100000000

# 定义辅助变量 y_im 和 y_dm
y_im = m.addMVar(19, vtype=GRB.BINARY, name="y_im")
y_dm = m.addMVar(19, vtype=GRB.BINARY, name="y_dm")
# 定义大M辅助变量 y_bigM
y_bigM = m.addMVar(19, vtype=GRB.BINARY, name="y_bigM")

M1 = m.addMVar((cdc_cnt_1,s_cnt), vtype=GRB.BINARY,name="M1")
M2 = m.addMVar((1,rdc_cnt), vtype=GRB.BINARY,name="M2")
M3 = m.addMVar((cdc_cnt_1,s_cnt), vtype=GRB.BINARY,name="M3")
M4 = m.addMVar((1,rdc_cnt), vtype=GRB.BINARY,name="M4")
#O = m.addMVar(dc_cnt, vtype=GRB.BINARY,name="O")
Z = m.addMVar(dc_cnt, vtype=GRB.BINARY,name="Z")
#Q_out=m.addMVar(s_cnt, vtype=GRB.BINARY,name="Q_out")#指示变量
# 更新变量环境
m.update()

tmp1=[j for j in range(19)]
tmp2=[j for j in range(19)]
# 定义辅助变量 tmp1 和 tmp2
for i in range(19):
    tmp1[i] = gp.quicksum(M1[i,j] * T_dts_1[i,j] for j in range(106))
for i in range(19):
    tmp2[i] = gp.quicksum(M3[i,j] * T_dts_1[i,j] for j in range(106))

##最大处置量
for i in range(0,20):
    Cap_r=cap_lim(M1,M3,Q_im,Q_dm)
    m.addConstr(Cap_r[i] <= Cap[i])

##cdc必开
#m.addConstr(O[6]==1)
#m.addConstr(O[10]==1)

##对应一个仓库
m.addConstrs((gp.quicksum(M1[:, i]) == 1 for i in range(106)), "Store_1")
m.addConstrs((gp.quicksum(M3[:, i]) == 1 for i in range(106)), "Store_3")

##M1 & M2约束
"""
def rdc_cdc_constraint(model, k, M_1, M_2, M):
    for i in range(18):
        # 去除第k行的约束表达式
        M_1_rdc = gp.quicksum(M_1[j, i] for j in range(19) if j != k)
        
        # 定义二进制变量，用于条件判断
        binary_var = model.addVar(vtype=GRB.BINARY, name=f"binary_var_{i}")

        # 约束1: M_1_rdc > 0 implies M_2[0, i] = 1 (实现 M_2[0, i] >= binary_var)
        model.addConstr(M_2[0, i] >= binary_var, f"rdc_cdc_cond1_{i}")

        # 约束2: M_1_rdc <= 0 implies M_2[0, i] = 0 (实现 binary_var <= M_1_rdc / M)
        model.addConstr(binary_var * M >= M_1_rdc, f"rdc_cdc_cond2_{i}")

        # 约束3: 当M_1_rdc为0时，M_2[0, i]必须为0
        model.addConstr(binary_var <= M_2[0, i], f"rdc_cdc_cond3_{i}")

rdc_cdc_constraint(m, k=5, M_1=M1, M_2=M2, M=1000)
rdc_cdc_constraint(m, k=5, M_1=M3, M_2=M4, M=1000)
"""
def rdc_cdc_constraint(model, k, M_1, M_2, M):
    M_1=M_1.tolist()
    M_1=np.array(M_1)
    M_1_dc = np.delete(M_1,k,axis=0)
    for i in range(18):
        # 去除第k行的约束表达式
        # M_1_rdc[i] = quicksum((M_1[j, n] for n in range(106)) for j in range(19) if j != k
        M_1_rdc = gp.quicksum(M_1_dc[i, j] for j in range(106))

        # 约束1: 当 M_1_rdc[i] > 0 时，M_2[0, i] = 1
        model.addConstr(M_2[0, i] * M >= M_1_rdc, f"rdc_cdc_cond1_{i}")
        
        # 约束2: 当 M_1_rdc[i] == 0 时，M_2[0, i] = 0
        model.addConstr(M_2[0, i] <= M_1_rdc / M, f"rdc_cdc_cond2_{i}")

rdc_cdc_constraint(m, k=6, M_1=M1, M_2=M2, M=1000)
rdc_cdc_constraint(m, k=10, M_1=M3, M_2=M4, M=1000)


#时间约束
for i in range(19):
    m.addConstr(y_im[i] <= tmp1[i] / 600)
    m.addConstr(y_dm[i] <= tmp2[i] / 600)
    m.addConstr(y_im[i] >= (tmp1[i] / 600) - BigM * (1 - y_bigM[i]))
    m.addConstr(y_dm[i] >= (tmp2[i] / 600) - BigM * (1 - y_bigM[i]))

# 定义 Q_out
Q_out = gp.quicksum(y_im[i] * Q_im[0,i] + y_dm[i] * Q_dm[0,i] for i in range(19))
# 添加时效满足率条件
m.addConstr(Q_out <= 0.05 * Q_all)

# 创建目标函数
m.setObjective(cal_all(M1,M2,M3,M4,O,Z,Of,Pf,L_ctd,L_dts_1,L_dts_2),GRB.MINIMIZE)

# 求解规划模型
m.optimize()
# 输出结果

# 输出结果
if m.status == GRB.OPTIMAL:
    print('Optimal Objective Value:', m.ObjVal)
    
    # 将结果转为pandas DataFrame
    M1_df = pd.DataFrame(M1.X)
    M2_df = pd.DataFrame(M2.X)
    M3_df = pd.DataFrame(M3.X)
    M4_df = pd.DataFrame(M4.X)
    #O_df = pd.DataFrame(O.X)
    Z_df = pd.DataFrame(Z.X)
    
    # 创建一个Excel writer对象
    with pd.ExcelWriter('optimization_results.xlsx') as writer:
        M1_df.to_excel(writer, sheet_name='M1', index=False)
        M2_df.to_excel(writer, sheet_name='M2', index=False)
        M3_df.to_excel(writer, sheet_name='M3', index=False)
        M4_df.to_excel(writer, sheet_name='M4', index=False)
        #O_df.to_excel(writer, sheet_name='O', index=False)
        Z_df.to_excel(writer, sheet_name='Z', index=False)
        
    print("Results saved to 'result1.xlsx'")
else:
    print('No optimal solution found.')

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 381 rows, 4141 columns and 23892 nonzeros
Model fingerprint: 0xf6ff9401
Model has 7844 quadratic objective terms
Variable types: 0 continuous, 4141 integer (4141 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+08]
  Objective range  [2e-01, 2e+06]
  QObjective range [5e-02, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+08]
Found heuristic solution: objective 1.619001e+07
Presolve removed 381 rows and 4141 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: